## 2.3 다중 선형 회귀 (Multivariable Linear regression)
  

02-01에서 배운 x가 한 개인 선형 회귀를 단순 선형 회귀(Simple Linear regression)이라고 한다. 이와 다르게 다수의 x로부터 y를 예측하는 선형 회귀를 다중 선형 회귀라고 한다.

### 2.3.1 데이터에 대한 이해 (Data Definition)

3개의 퀴즈 점수와 최종 점수가 저장된 데이터가 주어진다. 이 데이터를 이용항 3개의 퀴즈 점수로부터 최종 점수를 예측하는 모델을 구축한다. 독립 변수 x의 개수가 3개이므로 이를 수식으로 표현하면 다음같이 주어진다.  
  

>H(x) = w_1 * x_1 + w_2 * x_2 + w_3 * x_3

![훈련데이터.png](../image/훈련데이터.png)

### 2.3.2 파이토치로 구현하기

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [13]:
# 훈련 데이터 / x가 3개이므로 x를 3개 선언
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [80], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.294 w2: 0.294 w3: 0.290 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.688 w2: 0.681 w3: 0.658 b: 0.008 Cost: 5.754573
Epoch  200/1000 w1: 0.697 w2: 0.684 w3: 0.645 b: 0.008 Cost: 5.512386
Epoch  300/1000 w1: 0.707 w2: 0.686 w3: 0.634 b: 0.008 Cost: 5.281667
Epoch  400/1000 w1: 0.715 w2: 0.689 w3: 0.622 b: 0.008 Cost: 5.061868
Epoch  500/1000 w1: 0.724 w2: 0.691 w3: 0.611 b: 0.008 Cost: 4.852424
Epoch  600/1000 w1: 0.733 w2: 0.693 w3: 0.600 b: 0.008 Cost: 4.652705
Epoch  700/1000 w1: 0.741 w2: 0.695 w3: 0.589 b: 0.009 Cost: 4.462287
Epoch  800/1000 w1: 0.750 w2: 0.697 w3: 0.579 b: 0.009 Cost: 4.280604
Epoch  900/1000 w1: 0.758 w2: 0.699 w3: 0.569 b: 0.009 Cost: 4.107294
Epoch 1000/1000 w1: 0.766 w2: 0.700 w3: 0.559 b: 0.009 Cost: 3.941866


### 2.3.3 벡터와 행렬 연산으로 바꾸기

x가 3개였기에 위의 코드는 우리가 직접 구현했으나 만약 x가 1000개이면 전부 선언하기엔 너무 비효율적이다. 이를 해결하기 위해 행렬 곱셈 연산을 사용한다. 밑의 사진은 H(x)를 행렬의 곱셈으로 표현한 것이다.

![행렬곱셈.png](../image/행렬곱셈.png)

### 2.3.4 행렬 연산을 고려하여 파이토치로 구현하기

In [14]:
# 데이터를 행렬로 선언
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

In [15]:
print(x_train.shape) # 5 x 3 행렬
print(y_train.shape) # 5 x 1 행렬

torch.Size([5, 3])
torch.Size([5, 1])


In [16]:
# 가중치와 편향 선언
W = torch.zeros((3, 1), requires_grad=True) # x_train과 행렬곱이 가능해야 한다
b = torch.zeros(1, requires_grad=True)

In [17]:
# 가설 설정 / 추후 x의 개수가 늘어나도 식을 수정할 필요가 없다
hypothesis = x_train.matmul(W) + b

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

In [18]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    hypothesis = x_train.matmul(W) + b
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
            epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
        ))

Epoch    0/1000 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch  100/1000 hypothesis: tensor([154.0433, 185.0925, 175.8312, 198.5701, 141.2221]) Cost: 5.754568
Epoch  200/1000 hypothesis: tensor([154.0278, 185.0649, 175.9335, 198.5128, 141.2284]) Cost: 5.512375
Epoch  300/1000 hypothesis: tensor([154.0120, 185.0385, 176.0329, 198.4569, 141.2353]) Cost: 5.281663
Epoch  400/1000 hypothesis: tensor([153.9960, 185.0133, 176.1295, 198.4022, 141.2426]) Cost: 5.061868
Epoch  500/1000 hypothesis: tensor([153.9797, 184.9892, 176.2233, 198.3488, 141.2504]) Cost: 4.852397
Epoch  600/1000 hypothesis: tensor([153.9632, 184.9662, 176.3143, 198.2966, 141.2586]) Cost: 4.652731
Epoch  700/1000 hypothesis: tensor([153.9465, 184.9442, 176.4028, 198.2456, 141.2672]) Cost: 4.462276
Epoch  800/1000 hypothesis: tensor([153.9296, 184.9232, 176.4888, 198.1958, 141.2762]) Cost: 4.280613
Epoch  900/1000 hypothesis: tensor([153.9126, 184.9032, 176.5724, 198.1471, 141.2855]) Cost: 4.107294
Epoch 